In [ ]:
import os
import cv2
import numpy as np
import subprocess
import json

# Path utama
openpose_path = os.path.abspath("openpose")  # Sesuaikan path OpenPose
image_dir = os.path.abspath("../images")  # Folder gambar asli
flipped_dir = os.path.abspath("../flipped_images")  # Folder hasil flip
json_output_dir = os.path.abspath("../output_json")  # Folder JSON dari OpenPose

# Buat folder jika belum ada
os.makedirs(flipped_dir, exist_ok=True)
os.makedirs(json_output_dir, exist_ok=True)

# Ambil file gambar dari folder `images/`
image_files = [f for f in os.listdir(image_dir) if f.lower().endswith((".jpg", ".jpeg", ".png"))]

if not image_files:
    print("⚠️ Tidak ada gambar di folder `images/`.")
    exit()

# Pilih gambar pertama
input_image_path = os.path.join(image_dir, image_files[0])
file_name, file_ext = os.path.splitext(image_files[0])

# Tentukan output JSON
output_json_path = os.path.join(json_output_dir, f"{file_name}_keypoints.json")

# Jalankan OpenPose untuk mendeteksi keypoints
cmd = [
    "bin\\OpenPoseDemo.exe",
    "--image_dir", image_dir,
    "--write_json", json_output_dir,
    "--display", "0",
    "--render_pose", "0",
    "--model_pose", "BODY_25",
    "--number_people_max", "1"
]

print("Mendeteksi pose dengan OpenPose...")

# Jalankan OpenPose
result = subprocess.run(cmd, shell=True, capture_output=True, text=True, cwd=openpose_path)

# Cek jika JSON hasil OpenPose tersedia
if not os.path.exists(output_json_path):
    print("❌ Gagal mendeteksi pose! JSON tidak ditemukan.")
    exit()

# Baca JSON hasil OpenPose
with open(output_json_path, 'r') as f:
    pose_data = json.load(f)
    keypoints = pose_data['people'][0]['pose_keypoints_2d'] if pose_data['people'] else None

# Fungsi untuk mengambil koordinat keypoints
def get_coords(index):
    return (keypoints[index * 3], keypoints[index * 3 + 1]) if keypoints and keypoints[index * 3 + 2] > 0 else None

# Ambil titik hidung dan bahu
nose = get_coords(0)  # Hidung
left_shoulder = get_coords(2)  # Bahu kiri
right_shoulder = get_coords(5)  # Bahu kanan

if nose and left_shoulder and right_shoulder:
    center_shoulder_x = (left_shoulder[0] + right_shoulder[0]) / 2

    # Jika hidung di kiri dari bahu tengah, berarti menghadap kiri → Perlu di-flip
    if nose[0] < center_shoulder_x:
        print("🔄 Menghadap kiri → Flip ke kanan")
        img = cv2.imread(input_image_path)
        flipped_img = cv2.flip(img, 1)  # Flip horizontal
        flipped_image_path = os.path.join(flipped_dir, f"{file_name}_flipped{file_ext}")
        cv2.imwrite(flipped_image_path, flipped_img)
        print(f"✅ Gambar yang sudah di-flip disimpan di: {flipped_image_path}")
    else:
        print("✅ Menghadap kanan → Tidak perlu flip")
else:
    print("⚠️ Gagal mendeteksi posisi wajah dan bahu!")
